<a href="https://colab.research.google.com/github/BishwaKandel/CollegeRS/blob/intent/intent_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Classification using mBERT
This notebook demonstrates how to train an intent classification model using the multilingual BERT (mBERT) model from Huggingface Transformers.

## 1. Install Required Libraries

In [17]:
!pip install transformers datasets scikit-learn torch numpy==1.26.4 nlaug

ERROR: Could not find a version that satisfies the requirement nlaug (from versions: none)
ERROR: No matching distribution found for nlaug


In [19]:
import json
import pandas as pd
import numpy as np
from pprint import pprint
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
    EarlyStoppingCallback
)

In [20]:
from google.colab import files

print("Please upload your intent.json file:")
uploaded = files.upload()

# Load the uploaded intent.json file
intent_file = list(uploaded.keys())[0]  # Get the first uploaded file
with open(intent_file, 'r', encoding='utf-8') as f:
    intents = json.load(f)

print(f"✅ {intent_file} loaded successfully!")
print(f"Number of intents: {len(intents.get('intents', []))}")
pprint(intents)

Please upload your intent.json file:


Saving intent.json to intent (2).json
✅ intent (2).json loaded successfully!
Number of intents: 38
{'intents': [{'examples': ['What courses are offered?',
                           'List all available courses',
                           'Show me the courses',
                           'Which branches do you have?',
                           'Available academic programs?'],
              'intent': 'Course_list'},
             {'examples': ['Tell me about the Mechanical department',
                           'Details of the Civil department',
                           'What does the Computer department offer?',
                           'Give me information about Electrical department',
                           'Explain the departments in your college'],
              'intent': 'Department_info'},
             {'examples': ['Who teaches in Electrical Engineering?',
                           'Tell me about the computer faculty',
                           'Who are the faculty me

## 3. Prepare Data

In [21]:
# Convert examples to DataFrame
texts = []
labels = []

for intent in intents['intents']:
    for example in intent['examples']:
        texts.append(example)
        labels.append(intent['intent'])

df = pd.DataFrame({'text': texts, 'label': labels})
df.to_csv('intent_data.csv', index=False)
print(f"Dataset created with {len(df)} examples")
df.head()

Dataset created with 190 examples


,text,label
0,What courses are offered?,Course_list
1,List all available courses,Course_list
2,Show me the courses,Course_list
3,Which branches do you have?,Course_list
4,Available academic programs?,Course_list


## 4. Encode Labels and Create Dataset

In [22]:
# Encode labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

print(f"Number of unique intents: {len(label_encoder.classes_)}")
print(f"Intent classes: {list(label_encoder.classes_)}")

# Create dataset
dataset = Dataset.from_pandas(df[['text', 'label_encoded']])
dataset = dataset.rename_column('label_encoded', 'label')
dataset = dataset.train_test_split(test_size=0.2)

print(f"Train set size: {len(dataset['train'])}")
print(f"Test set size: {len(dataset['test'])}")

Number of unique intents: 38
Intent classes: ['Academic_calendar', 'Admission_deadline', 'Admission_officer_contact', 'Admission_process', 'Alumni_network', 'Anti_ragging_measures', 'Application_status', 'Campus_facilities', 'Campus_visit_schedule', 'Clarification_needed', 'College_location', 'Contact_information', 'Course_list', 'Department_fee_query', 'Department_info', 'Department_scholarship_query', 'Direct_admission', 'Document_required', 'Eligibility_criteria', 'Entrance_exam_info', 'Exam_schedule', 'Extracurricular_activities', 'Faculty_info', 'Fee_structure', 'Hostel_fee', 'Hostel_info', 'Internship_opportunities', 'Nearby_locations', 'Payment_methods', 'Placement_info', 'Ragging_policy', 'Repeat_question', 'Scholarship_application', 'Scholarship_criteria', 'Student_support_services', 'Syllabus_query', 'Transport_facility', 'Unknown_query']
Train set size: 152
Test set size: 38


## 5. Tokenization

In [23]:
# Initialize tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

# Tokenize dataset
dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print("Tokenization completed!")

Map:   0%|          | 0/152 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Tokenization completed!


## 6. Model Initialization

In [24]:
# Initialize model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_encoder.classes_)
)

print(f"Model initialized with {len(label_encoder.classes_)} output labels")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model initialized with 38 output labels



# Freeze lower BERT layers (e.g., first 6)

In [25]:
for name, param in model.bert.named_parameters():
    if name.startswith("encoder.layer.") and int(name.split(".")[2]) < 6:
        param.requires_grad = False
print("Lower layers frozen for regularization.")

Lower layers frozen for regularization.


## 7. Training Configuration

In [26]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./intent_model',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='epoch',
    eval_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True
)

# Metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': acc, 'f1': f1}

print("Training configuration set!")

Training configuration set!


## 8. Model Training

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Start training
print("Starting training...")
trainer.train()
print("Training completed!")

/tmp/ipython-input-27-4040003251.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,3.652600,3.651603,0.000000,0.000000
2,3.550100,3.682700,0.000000,0.000000
3,3.449100,3.657552,0.026316,0.013158
4,3.425300,3.613039,0.078947,0.070175
5,3.310500,3.581799,0.052632,0.035088


## 9. Model Evaluation

In [12]:
# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

Evaluation Results:
eval_loss: 3.6063
eval_accuracy: 0.0789
eval_f1: 0.0497
eval_runtime: 17.9076
eval_samples_per_second: 2.1220
eval_steps_per_second: 0.2790
epoch: 3.0000


## 10. Save Model

In [13]:
# Save the trained model and tokenizer
model.save_pretrained('intent_classifier_model')
tokenizer.save_pretrained('intent_classifier_model')

# Save label encoder
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("Model and tokenizer saved successfully!")

Model and tokenizer saved successfully!


## 11. Inference Example

In [14]:
# Load the classification pipeline
classifier = pipeline(
    'text-classification',
    model='intent_classifier_model',
    tokenizer='intent_classifier_model'
)

# Test sentences
test_sentences = [
    'hello there',
    'tell me about the electrical branch',
    'what are the computer fees',
    'goodbye',
    'thank you very much'
]

print("Predictions:")
print("-" * 50)

for sentence in test_sentences:
    result = classifier(sentence)

    # Extract label index
    label_str = result[0]['label']
    if 'LABEL_' in label_str:
        label_idx = int(label_str.split('_')[-1])
    else:
        label_idx = int(label_str)

    # Get original intent name
    predicted_intent = label_encoder.inverse_transform([label_idx])[0]
    confidence = result[0]['score']

    print(f"Input: '{sentence}'")
    print(f"Predicted Intent: {predicted_intent}")
    print(f"Confidence: {confidence:.4f}")
    print()

print("Inference completed!")

Device set to use cpu


Predictions:
--------------------------------------------------
Input: 'hello there'
Predicted Intent: Eligibility_criteria
Confidence: 0.0389

Input: 'tell me about the electrical branch'
Predicted Intent: Placement_info
Confidence: 0.0555

Input: 'what are the computer fees'
Predicted Intent: Extracurricular_activities
Confidence: 0.0487

Input: 'goodbye'
Predicted Intent: Admission_officer_contact
Confidence: 0.0360

Input: 'thank you very much'
Predicted Intent: Placement_info
Confidence: 0.0415

Inference completed!


## 12. Interactive Prediction

In [15]:
# Interactive prediction function
def predict_intent(text):
    result = classifier(text)
    label_str = result[0]['label']

    if 'LABEL_' in label_str:
        label_idx = int(label_str.split('_')[-1])
    else:
        label_idx = int(label_str)

    predicted_intent = label_encoder.inverse_transform([label_idx])[0]
    confidence = result[0]['score']

    return predicted_intent, confidence

# Example usage
user_input = "How much does it cost to study here?"
intent, conf = predict_intent(user_input)
print(f"User: {user_input}")
print(f"Predicted Intent: {intent} (Confidence: {conf:.4f})")

User: How much does it cost to study here?
Predicted Intent: Campus_facilities (Confidence: 0.0443)


In [ ]:
# Update libraries to resolve potential conflicts
!pip install --upgrade datasets transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 896.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1